In [1]:
%pwd

'c:\\Users\\prita\\Downloads\\law_enforcement\\research'

In [2]:
import os
os.chdir("../")

In [3]:
%pwd

'c:\\Users\\prita\\Downloads\\law_enforcement'

In [4]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

c:\Users\prita\Downloads\law_enforcement\.legalhelpvenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
def load_data(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )

    documents = loader.load()
    return documents

In [6]:
extracted_data = load_data(data="Data/")

In [7]:
extracted_data

[Document(metadata={'producer': 'Microsoft® Word 2019', 'creator': 'Microsoft® Word 2019', 'creationdate': '2025-01-14T16:54:46+05:30', 'author': 'Lenovo', 'moddate': '2025-01-14T16:54:46+05:30', 'source': 'Data\\Bharatiya Nagarik Suraksha Sanhita.pdf', 'total_pages': 279, 'page': 0, 'page_label': '1'}, page_content='1 \n \nTHE BHARATIYA NAGARIK SURAKSHA SANHITA, 2023 \n_____________ \nARRANGEMENT OF SECTIONS \n__________ \nCHAPTER I \nPRELIMINARY \nSECTIONS \n1. Short title, extent and commencement. \n2. Definitions. \n3. Construction of references. \n4. Trial of offences under Bharatiya Nyaya Sanhita, 2023 and other laws. \n5. Saving. \nCHAPTER II \nCONSTITUTION OF CRIMINAL COURTS AND OFFICES \n \n6. Classes of Criminal Courts. \n7. Territorial divisions. \n8. Court of Session. \n9. Courts of Judicial Magistrates. \n10. Chief Judicial Magistrate and Additional Chief Judicial Magistrate, etc. \n11. Special Judicial Magistrates. \n12. Local Jurisdiction of Judicial Magistrates. \n13.  

In [8]:
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [9]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 4303


In [10]:
text_chunks

[Document(metadata={'producer': 'Microsoft® Word 2019', 'creator': 'Microsoft® Word 2019', 'creationdate': '2025-01-14T16:54:46+05:30', 'author': 'Lenovo', 'moddate': '2025-01-14T16:54:46+05:30', 'source': 'Data\\Bharatiya Nagarik Suraksha Sanhita.pdf', 'total_pages': 279, 'page': 0, 'page_label': '1'}, page_content='1 \n \nTHE BHARATIYA NAGARIK SURAKSHA SANHITA, 2023 \n_____________ \nARRANGEMENT OF SECTIONS \n__________ \nCHAPTER I \nPRELIMINARY \nSECTIONS \n1. Short title, extent and commencement. \n2. Definitions. \n3. Construction of references. \n4. Trial of offences under Bharatiya Nyaya Sanhita, 2023 and other laws. \n5. Saving. \nCHAPTER II \nCONSTITUTION OF CRIMINAL COURTS AND OFFICES \n \n6. Classes of Criminal Courts. \n7. Territorial divisions. \n8. Court of Session. \n9. Courts of Judicial Magistrates.'),
 Document(metadata={'producer': 'Microsoft® Word 2019', 'creator': 'Microsoft® Word 2019', 'creationdate': '2025-01-14T16:54:46+05:30', 'author': 'Lenovo', 'moddate': '2

In [11]:
from langchain.embeddings import HuggingFaceEmbeddings

In [12]:
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [13]:
embeddings = download_hugging_face_embeddings()

C:\Users\prita\AppData\Local\Temp\ipykernel_17184\4043855499.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [14]:
from dotenv import load_dotenv
load_dotenv()

True

In [15]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.environ.get("HUGGINGFACEHUB_API_TOKEN")

In [19]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "chanakyabot"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

{
    "name": "chanakyabot",
    "metric": "cosine",
    "host": "chanakyabot-v98f33x.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [20]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

c:\Users\prita\Downloads\law_enforcement\.legalhelpvenv\Lib\site-packages\langchain_pinecone\__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore


In [21]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [22]:
docsearch

In [23]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [24]:
retrieved_docs = retriever.invoke("What is Right of citizenship?")

In [25]:
retrieved_docs

[Document(id='03c8e7d5-14eb-4998-8089-559bcf8146a9', metadata={'creationdate': '', 'creator': 'PyPDF', 'moddate': '2024-07-01T11:20:33+00:00', 'page': 35.0, 'page_label': '36', 'producer': 'iLovePDF', 'source': 'Data\\Bharatiya Nyaya Sanhita.pdf', 'total_pages': 402.0}, page_content='be deemed to be a citizen of India by virtue of article 6 or article 8, if he has voluntarily acquired the citizenship of any foreign State.10. Continuance of the rights of citizenship.—Every person who is or is deemed to be a citizen of India under any of the foregoing provisions of this Part shall, subject to the provisions of any law that may be made by Parliament, continue to be such citizen.11. Parliament to regulate the right of citizenship by law.—Nothing in the foregoing provisions of'),
 Document(id='b22beadf-f797-498b-b2e4-0d2b508258c2', metadata={'creationdate': '', 'creator': 'PyPDF', 'moddate': '2024-07-01T11:20:33+00:00', 'page': 34.0, 'page_label': '35', 'producer': 'iLovePDF', 'source': 'Da

In [26]:
# from langchain_openai import OpenAI
# llm = OpenAI(temperature=0.4, max_tokens=500)

In [27]:
from langchain_community.llms import HuggingFaceEndpoint
import os

llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.2", # A powerful open-source model
    temperature=0.5,
    max_new_tokens=512
)

C:\Users\prita\AppData\Local\Temp\ipykernel_17184\2758696550.py:4: LangChainDeprecationWarning: The class `HuggingFaceEndpoint` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  llm = HuggingFaceEndpoint(


In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are a helpful and precise assistant for question-answering tasks. "
    "Your main task is to answer the user's question accurately using ONLY the provided context. "
    "Follow these rules strictly:\n"
    "1. **Analyze the Context:** Carefully read the provided context below to find the information needed to answer the question.\n"
    "2. **Ground Your Answer:** Base your entire answer on the facts found in the context. Do not use any of your own external knowledge or make assumptions.\n"
    "3. **Handle Missing Information:** If the context does not contain the information to answer the question, you MUST say: 'I'm sorry, but the provided documents do not contain the answer to that question.' Do not try to guess.\n"
    "4. **Be Concise:** Keep the answer to a maximum of three sentences.\n"
    "5. **Do Not Mention the Context:** Do not say 'According to the context...' or 'The context states...'. Answer the question directly as if you are the expert.\n\n"
    "--- CONTEXT ---"
    "\n{context}\n"
    "--- END OF CONTEXT ---"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [ ]:
# question_answer_chain = create_stuff_documents_chain(llm, prompt)
# rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
from langchain_community.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM

# Choose a model (e.g., a smaller, more manageable one)
# model_id = "gpt2" 
# model_id = "distilgpt2"
model_id = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

# pipe_for_gpt_model = pipeline(
#     "text-generation", model=model, tokenizer=tokenizer, max_new_tokens=256
# )

pipe_for_google_model = pipeline(
    "text2text-generation", model=model, tokenizer=tokenizer, max_new_tokens=256
)
llm = HuggingFacePipeline(pipeline=pipe_for_google_model)

# The rest of your code for creating the chain remains the same!
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

Device set to use cpu


In [38]:
response = rag_chain.invoke({"input": "What is Right for Protection of life and personal liberty?"})
print(response["answer"])

No person shall be deprived of his life or personal liberty except according to procedure established by law.2[21A. Right to education.—The State shall provide free and compulsory education to all children of the age of six to fourteen years in such manner as the State may, by law, determine.]22. Protection against arrest and detention in certain cases.—(1) No person who is arrested shall be detained in custody without being informed, as soon as may be, of the grounds for such arrest in respect of conviction for offences.


In [39]:
response = rag_chain.invoke({"input": "What is Machine Learning"})
print(response["answer"])

I can not help you with this.
